### 지니차트 Top200

In [1]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

##### 1. 데이터 가져오기

In [2]:
url = 'https://www.genie.co.kr/chart/top200'
result = requests.get(url)
result.text

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [3]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
result = requests.get(url, headers=header)
soup = BeautifulSoup(result.text, 'html.parser')

#### 2. 원하는 데이터 찾기

In [4]:
trs = soup.select('tr.list')
len(trs)

50

In [5]:
# 개발자 도구에서 찾아주는 Selector
trs = soup.select('#body-content > div.newest-list > div > table > tbody > tr')

In [6]:
len(trs)

50

#### 3. 하나의 정보를 선택해서 원하는 정보 추출

In [7]:
tr = trs[0]

In [8]:
tr.select_one('.number').get_text()

'1\n                                        \n                                    \n유지\n\n'

In [9]:
rank = tr.select_one('.number').get_text().split('\n')[0].strip

In [10]:
# img url
img = tr.select_one('.cover > img')['src']
img

'//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/083/665/559/83665559_1681116738103_1_140x140.JPG/dims/resize/Q_80,0'

In [11]:
#title
tr.select_one('.title.ellipsis').get_text().strip()

'I AM'

In [12]:
# 19금 노래 제목
tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()

'I AM'

In [13]:
# 위의 두 코드를 합치면
# 방법1
tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
# 방법2
try:
    title = tr.select_one('.title.ellipsis').get_text().strip()
except:
    title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()

In [14]:
# artists
artist = tr.select_one('.artist.ellipsis').string.strip()
artist

'IVE (아이브)'

In [15]:
# album
album = tr.select_one('.albumtitle.ellipsis').text.strip()
album

"I've IVE"

#### 4. 한페이지내에 있는 정보

In [16]:
# 방법1. 딕셔너리의 리스트로 가져오기
line = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0].strip()
    img = 'https:' + tr.select_one('.cover > img')['src']
    title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
    artist = tr.select_one('.artist.ellipsis').string.strip()
    album = tr.select_one('.albumtitle.ellipsis').text.strip()
    line.append({'rank':rank, 'img':img, 'title':title, 'artist':artist, 'album':album})

In [17]:
# 방법2. 개별리스트로 가져오기
rank_list, img_list, title_list, artist_list, album_list = [],[],[],[],[]
for tr in trs:
    rank_list.append(tr.select_one('.number').get_text().split('\n')[0].strip)
    img_list.append(tr.select_one('.cover > img')['src'])
    title_list.append(tr.select_one('.title.ellipsis').get_text().strip())
    artist_list.append(tr.select_one('.artist.ellipsis').string.strip())
    album_list.append(tr.select_one('.albumtitle.ellipsis').text.strip())

In [18]:
df1 = pd.DataFrame(line)
df1.head()

,rank,img,title,artist,album
0,1,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,I AM,IVE (아이브),I've IVE
1,2,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Kitsch,IVE (아이브),I've IVE
2,3,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,손오공,세븐틴 (SEVENTEEN),SEVENTEEN 10th Mini Album 'FML'
3,4,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,꽃,지수 (JISOO),ME
4,5,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,F*ck My Life,세븐틴 (SEVENTEEN),SEVENTEEN 10th Mini Album 'FML'


In [19]:
df2 = pd.DataFrame({
    'rank':rank_list,'img':img_list,'title':title_list,'artist':artist_list,'album':album_list
})

In [20]:
df2.head()

,rank,img,title,artist,album
0,<built-in method strip of str object at 0x0000...,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/083/665/...,I AM,IVE (아이브),I've IVE
1,<built-in method strip of str object at 0x0000...,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/083/665/...,Kitsch,IVE (아이브),I've IVE
2,<built-in method strip of str object at 0x0000...,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/083/762/...,손오공,세븐틴 (SEVENTEEN),SEVENTEEN 10th Mini Album 'FML'
3,<built-in method strip of str object at 0x0000...,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/083/687/...,꽃,지수 (JISOO),ME
4,<built-in method strip of str object at 0x0000...,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/083/762/...,F*ck My Life,세븐틴 (SEVENTEEN),SEVENTEEN 10th Mini Album 'FML'


#### 5. 에러 처리

#### 6. 모든 페이지에 대해서 처리하기

In [21]:
from datetime import datetime
now = datetime.now()

In [22]:
ymd = now.strftime('%y%m%d')    # ymd = f'{now.year}{now,mondth:02d}{now.day02d}'
hh = now.strftime('%H')         # hh f'{now.hour:02d}'
ymd, hh

('230425', '08')

In [26]:
from tqdm import tqdm
lines = []
base_url = 'https://www.genie.co.kr/chart/top200?ditc=D&rtm=Y'
for page in tqdm(range(1,5)):
    url = f'{base_url}&ymd={ymd}&hh={hh}%pg={page}'
    result = requests.get(url, headers=header)
    soup = BeautifulSoup(result.text, 'html.parser')
    trs = soup.select('tr.list')
    for tr in trs:
        rank = tr.select_one('.number').get_text().split('\n')[0].strip()
        img = 'https:' + tr.select_one('.cover > img')['src']
        title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
        artist = tr.select_one('.artist.ellipsis').string.strip()
        album = tr.select_one('.albumtitle.ellipsis').text.strip()
        lines.append({'rank':rank, 'img':img, 'title':title, 'artist':artist, 'album':album})

100%|██████████| 4/4 [00:01<00:00,  2.48it/s]


#### 7. 데이터프레임 만들기

In [27]:
df = pd.DataFrame(lines)
df.head()

,rank,img,title,artist,album
0,1,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,I AM,IVE (아이브),I've IVE
1,2,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Kitsch,IVE (아이브),I've IVE
2,3,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,손오공,세븐틴 (SEVENTEEN),SEVENTEEN 10th Mini Album 'FML'
3,4,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,꽃,지수 (JISOO),ME
4,5,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,F*ck My Life,세븐틴 (SEVENTEEN),SEVENTEEN 10th Mini Album 'FML'


In [28]:
df.to_csv(f'data/{ymd}{hh}_지니차트.csv',index=False)